In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import re


#this copy uses padding to batch the data

In [2]:
# splitting data into names
data = open('babynames.txt', 'r').read()
data = data.lower()
chars = list(set(data))
vocab_size = len(chars)
pattern = r'[^ \n][^ \n]*\n'
target = re.findall(pattern, data)
pattern = r'[^\n][^ \n]+\n'
data = re.findall(pattern, data)
maxlen = len(max(data, key=len))

# Padding
for i in range(len(data)):
    print(i)
    print(data[i])
    print(target[i]+"s")
    assert(len(data[i])==len(target[i]))
    while len(data[i])<maxlen:
        data[i] += '\n'
        target[i] += '\n'

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}


# Reshaping data
data = [[char2int[char] for char in name] for name in data]
target = [[char2int[char] for char in name] for name in target]
data = torch.tensor(data)
target = torch.tensor(target)
data = F.one_hot(data, num_classes=vocab_size).float()
target = F.one_hot(target, num_classes=vocab_size).float()

0
 michael

michael
s


AssertionError: 

In [3]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim

        #Defining the layers
        self.rnn = nn.RNN(input_size, hidden_dim)   
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, input_seq, hidden):
        out, hidden = self.rnn(input_seq, hidden)    #input_seq=seq_len*h_in, out=seq_len*h_out
        out = self.fc(out)    #out=seq_len*output_size
        
        return out, hidden
    

In [4]:
# Define hyperparameters
lr=0.01

# Instantiate the model with hyperparameters
model = Model(input_size=vocab_size, output_size=vocab_size, hidden_dim=100)

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [5]:
def sample(out, hidden, size):
    print(out.shape)
    name=torch.zeros(size)
    for counter in range(size):
        #get a probability distribution from output
        prob = F.softmax(out, dim=1)
        
        #given prob generate a character randomly
        index = np.random.choice(vocab_size, prob)
        input_seq = F.one_hot(index, num_classes=vocab_size)
    
        name[count] = input_seq
        out, hidden = model(input_seq , hidden)        
    
    #reverse one hot encoding
    name = name.argmax(-1)
    
    #join characters
    name = ''.join(int2char[char] for char in name)    
    return name

def sample(size):
    name = torch.zeros(size, vocab_size)
    index = random.randint(0,len(data)-1)
    char = data[index]
    
    for counter in range(size):
        name[counter] = char
        char = char.view(1, vocab_size)
        out, hidden = model(char)
        
        #get a probability distribution from output
        prob = F.softmax(out, dim=0)
        
        #given prob generate a character randomly
        char = torch.multinomial(prob, num_samples=1)
        char = F.one_hot(char, num_classes=vocab_size).float()
            
    
    #reverse one hot encoding
    name = name.argmax(-1)
    name = name.numpy()
    #join characters
    sample = ''.join(int2char[integer] for integer in name)    
    return sample

In [6]:
#Training run
epoch=0
while True:  #epochs
    epoch += 1
    print('Epoch: {}.............'.format(epoch), end=' ')
    hidden = torch.zeros(1, hidden_dim)   
    for counter, input_seq in enumerate(data):
        optimizer.zero_grad() # Clears existing gradients from previous epoch
        hidden = hidden.detach() #Prevents backpropagation through the hidden state
        out, hidden = model(input_seq, hidden)
        loss = criterion(output, target[counter])
        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly
    
        # sample from the model now and then
        if counter % 4 == 0:
            name = sample(out[-1], hidden, 200)            
            print('----\n %s \n----' % (name, ))
            print("Loss: {:.4f}".format(loss.item()))
            

Epoch: 1............. 

NameError: name 'hidden_dim' is not defined